# Order a Basemap Using the Orders API & SDK via AOI & Quad ID

## Overview ##
---
In this notebook, you will learn how to order a [Planet Basemap](https://developers.planet.com/docs/data/visual-basemaps/) using your [Area of Interest](https://developers.planet.com/apis/orders/basemaps/#order-basemaps-by-area-of-interest-aoi) (AOI) and a [Quad ID](https://developers.planet.com/apis/orders/basemaps/#order-basemaps-by-quad-ids-and-deliver-to-cloud). We will place this order via Planet's [Orders API](https://developers.planet.com/apis/orders/) using our Planet Python [SDK](https://planet-sdk-for-python-v2.readthedocs.io/en/latest/python/sdk-guide/). 

1. Get a Basemap ID using either [Planet Explorer](https://developers.planet.com/docs/apps/explorer/) or the [Basemap Viewer](https://www.planet.com/basemaps/#/mosaic/). 
2. Create a JSON order packet with order parameters.
3. Set up a session with the Planet SDK, and place the order, [delivering]((https://developers.planet.com/apis/orders/delivery/)) the resulting Basemap to a Google Cloud Storage bucket. 
4. Repeat steps 1-3 using quad IDs. 

## Order a Basemap using an AOI
---
### First, find a Basemap to order
You can get a Basemap ID from Planet Explorer or the Basemap Viewer.

#### Option 1: Get a basemap ID from Planet Explorer
You'll need a basemap ID to identify what basemap to download.

To pick a basemap, you can open [Planet Explorer](https://www.planet.com/explorer/#) (you can learn more about Planet Explorer [here](https://developers.planet.com/docs/apps/explorer/)) and select a basemap from the dropdown menu in the lower left corner. (If you don't have access to basemaps, you'll see a message confirming in the lower right corner. Select "Get Access" to sign up for Basemaps or speak to your customer service manager.)

Once you've selected a Basemap, the Explorer updates to include that basemap in the view.

Select the search icon and time cadence, for example `monthly` or `quarterly`. Then select one of the resulting basemap IDs, such as `global_monthly_2022_01_mosaic`. That's the value you'll be passing into the Orders API. 

#### Option 2: Get a basemap ID from the Basemap Viewer

To pick a basemap, you can open the [Basemap Viewer](https://www.planet.com/basemaps/#/mosaic/) (you can learn more about the Basemap Viewer [here](https://developers.planet.com/docs/apps/basemapsviewer/)) and select a basemap from the left sidebar menu, using the filter to narrow down the basemap you want. 

Once you've selected a basemap, and selected the right arrow >, the basemap ID displays, such as `global_monthly_2022_01_mosaic`. That's the value you'll be passing into the Orders API.

### Import Planet and Related Packages

---

Make sure you have Planet's Python SDK properly downloaded. You can find out more about this [here](https://developers.planet.com/docs/pythonclient/). Find your [API key](https://developers.planet.com/quickstart/apis/).

Next set up a session by importing needed Python packages, pulling in your API Key, and make an initial request (to retrieve the Orders API parameters) to confirm a connection with the server.

Create a Google Cloud Platform (GCP) storage bucket with these [instructions](https://cloud.google.com/storage/docs/creating-buckets). This is where we are delivering the data. Creating the bucket will create the credentials for you, and you can convert it to Base64 using these [instructions](https://cloud.google.com/vision/docs/base64) and save this as an environment variable. 

In [81]:
import planet
import os
import copy
import requests
from requests.auth import HTTPBasicAuth
from urllib3.util.retry import Retry
from requests.adapters import HTTPAdapter
import asyncio

In [82]:
PL_API_KEY = os.environ.get('PL_API_KEY')
GCP_CREDENTIALS = os.environ.get('GCP_CREDENTIALS')
auth = planet.Auth.from_key(PL_API_KEY)

### Create an order packet
---
Package up the details of your order in a [JSON object](https://developers.planet.com/apis/orders/basemaps/#example-order-query-json-block) and make a POST request, passing in the Orders URL, your JSON, your API key, and the content-type header. We are [delivering](https://developers.planet.com/apis/orders/delivery/) this order to a Google Cloud Storage bucket. In this request, we are also using the tools [merge](https://developers.planet.com/apis/orders/tools/#merge) and [clip](https://developers.planet.com/apis/orders/tools/#clip).  ADD MOSAIC NAME HERE !!!!

In [49]:
BASEMAP_API_URL = 'https://api.planet.com/basemaps/v1/mosaics'


In [50]:
order_params = {
    "name": "basemap order with geometry",
    "source_type": "basemaps",
    "order_type": "partial",
    "products": [
        {
            "mosaic_name": "global_monthly_2022_01_mosaic",
            "geometry": {
                "type": "Polygon",
                "coordinates": [
                   [
                       [4.607406, 52.353994],
                       [4.680005, 52.353994],
                       [4.680005, 52.395523],
                       [4.607406, 52.395523],
                       [4.607406, 52.353994]
                   ]
                ]
            }
        }
    ],
    "delivery": {
        "google_cloud_storage": {
            "bucket": "devrel-notebooks",
            "credentials": GCP_CREDENTIALS,
            "path_prefix": "basemaps-to-cloud/"
        }
    }
}

### Create a session with SDK and poll for success

Here, we are creating an order using the SDK. Then we are waiting for the order to be delievered to our GCP storage bucket. Note that this can take some time.

In [51]:
async def create_and_deliver_order(order_params, client):
    '''Create an order and wait for it to delivered

    Parameters:
        order_params: An order request
        client: An Order client object
    '''
    with planet.reporting.StateBar(state='creating') as reporter:
        # Place an order to the Orders API
        order = await client.create_order(order_params)
        reporter.update(state='created', order_id=order['id'])
        # Wait while the order is being completed
        await client.wait(order['id'], callback=reporter.update_state,
                          max_attempts=0)

In [52]:
async with planet.Session() as ps:
    # The Orders API client
    client = ps.client('orders')
    # Create the order and deliver it to GCP
    await asyncio.gather(create_and_deliver_order(order_params, client))
    
    

00:21 - order 8f325f58-35f3-4918-87a3-e0c5ea7e660a - state: running


CancelledError: 

### Check your results

After a few simple steps, we have a basemap delivered to our Google Cloud Platform bucket using the Planet Python SDK.

## Use Basemaps API to get the Basemap and quad IDs
---

### What is a Quad ID? 

A quad ID is a unique identifer to access a square tile of imagery of the Earth's surface, whereas a Basemap is a combination of squares. 

Use the Basemap APIs to retrieve the basemap you want and the quad IDs you are looking for. You can get the base URL you'll need to communicate with the Basemap API service:

1. Go to the [Basemap API reference](https://developers.planet.com/docs/basemaps/reference/) at https://developers.planet.com/docs/basemaps/reference/. 
2. Under List Mosaics, select GET /mosaics.

A dropdown UI element appears with the URL to use:

https://api.planet.com/basemaps/v1/mosaics


In [53]:
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

In [54]:
auth = HTTPBasicAuth(PL_API_KEY, '')
session = requests.Session()

# Authenticate
session.auth = (PL_API_KEY, '')

In [55]:
def handle_pagination(session, url, params, key='items'):
    """Handle paginated URLs"""
    while True:
        response = session.get(url, params=params)
        response.raise_for_status()
        body = response.json()
        for item in body[key]:
            yield item

        if '_next' in body['_links']:
            url = body['_links']['_next']
        else:
            break

In [56]:
BASEMAP_API_URL = 'https://api.planet.com/basemaps/v1/mosaics'

# Wait and retry automatically on 429 "slow down" responses
retries = Retry(total=10, backoff_factor=1, status_forcelist=[429])
session.mount('https://', HTTPAdapter(max_retries=retries))


basemap_params = {
    'name__is': 'fusion_v120rc4_2022-11-04_mosaic'
}

session.headers.update({'content-type': 'application/json'})

In [57]:
mosaic, = handle_pagination(session, BASEMAP_API_URL, basemap_params, 'mosaics')

quad_params = {
    'bbox': ','.join(map(str, mosaic['bbox']))
}
quads_url = "{}/{}/quads".format(BASEMAP_API_URL, mosaic['id'])
quads = handle_pagination(session, quads_url, quad_params)

quad_ids = [item['id'] for item in quads if 'download' in item['_links']]


In [58]:
order_params = {
    "name": "basemap order with quad_ids",
    "source_type": "basemaps",
    "products": [
        {
            "mosaic_name": "fusion_v120rc4_2022-11-04_mosaic",
            "quad_ids": quad_ids
        }
    ],
    "tools": [
        {
            "bandmath": {
                "b1": "b1",
                "b2": "b2",
                "b3": "b3",
                "b4": "b4",
                "b5": "(b4-b3)/(b4+b3)",
                "pixel_type": "32R"
            }
        }
    ],
    "delivery": {
        "google_cloud_storage": {
            "bucket": "devrel-notebooks",
            "credentials": GCP_CREDENTIALS,
            "path_prefix": "basemaps-to-cloud/",
        }
    }
}

In [59]:
order_params_1 = copy.deepcopy(order_params)
order_params_2 = copy.deepcopy(order_params)

all_order_params = [order_params_1, order_params_2]
batched_quad_ids = [quad_ids[i:i + 100] for i in range(0, len(quad_ids), 100)]
[all_order_params[i]['products'][0].__setitem__('quad_ids', batched_quad_ids[i]) for i in range(len(batched_quad_ids))]

[None, None]

In [60]:
async with planet.Session() as ps:
    # The Orders API client
    client = ps.client('orders')
    # Create the order and deliver it to GCP
    await (create_and_deliver_order(order_params, client))
    
    

00:00 - order  - state: creating


BadQuery: {"field":null,"general":[{"message":"Unable to accept order: validation of input quads failed: Bad Request : {\"message\":\"Maximum number of IDS is 100\"}\n"}]}


#### Search for the mosaic ID to get its quad IDs

Declare the search string for `requests.get()`. You will use the dictionary you create with the mosaic name to pass to the requests.get() function.


This code sets the URL and headers for an HTTP request to Planet's Basemap API and sends a GET request with the specified search parameters. If the response returns the correct status, the JSON response is loaded into a Python dictionary with mosaics information from which we'll extract quad ID information.

This code retrieves the ID and bounding box coordinates of the mosaic we loaded above and uses these to construct an API request to retrieve the corresponding quad IDs for a specific area.

In [ ]:
mosaic_name = 'ps_biweekly_sen2_normalized_analytic_subscription_2023-04-17_2023-05-01_mosaic'
geometry = {
    "coordinates": [[[-93.2969252074559, 31.625920464644352],
                        [-91.12907701200592, 31.462828609443292],
                        [-89.55957445650489, 35.19041389553804],
                        [-84.17442809744674, 37.558687870257046],
                        [-90.25210159772769, 37.773797433818345],
                        [-92.93646531283778, 35.481589006307814],
                        [-93.2969252074559, 31.625920464644352]]],
    "type": "Polygon"
    }

In [86]:
def polygon_search(mosaic_name, geometry, base_url):

    retries = Retry(total=5, backoff_factor=0.2, status_forcelist=[429])
    session.mount('https://', HTTPAdapter(max_retries=retries))

    rv = session.get(f'{base_url}/mosaics', params={'name__is': mosaic_name})
    rv.raise_for_status()
    mosaic_id = rv.json()['mosaics'][0]['id']

    url = None
    while True:
        if url is None:
            url = f'{base_url}/mosaics/{mosaic_id}/quads/search'
            rv = session.post(url, json=geometry)
        else:
            rv = session.get(url)
        rv.raise_for_status()
        response = rv.json()

        for item in response['items']:
            yield item

        if '_next' in response['_links']:
            url = response['_links']['_next']
        else:
            break

    return quad_ids

In [88]:
quad_ids = []

for quad in polygon_search(mosaic_name, geometry, BASEMAP_API_URL):
    quad_ids.append(quad['id'])

### Create an order packet
---
Create an order packet with details including the mosaic name and quad IDs, and the tools and delivery method. In this example, we are using the tools [bandmath](https://developers.planet.com/apis/orders/tools/#band-math) and [reproject](https://developers.planet.com/apis/orders/tools/#reproject). 

#### Bandmath 

The bandmath tool allows you to apply band math expressions to the bands of your input files to produce derived outputs and indices for analysis. Note that `b5` in this example is being assigned to [NDVI](https://en.wikipedia.org/wiki/Normalized_difference_vegetation_index). 

In [ ]:
order_params = {
    "name": "basemap order with quad_ids",
    "source_type": "basemaps",
    "products": [
        {
            "mosaic_name": "ps_biweekly_sen2_normalized_analytic_subscription_2023-04-17_2023-05-01_mosaic",
            "quad_ids": quad_ids
        }
    ],
    "tools": [
        {
            "bandmath": {
                "b1": "b1",
                "b2": "b2",
                "b3": "b3",
                "b4": "b4",
                "b5": "(b4-b3)/(b4+b3)",
                "pixel_type": "32R"
            }
        }
    ],
    "delivery": {
        "google_cloud_storage": {
            "bucket": "devrel-notebooks",
            "credentials": GCP_CREDENTIALS,
            "path_prefix": "basemaps-to-cloud/",
        }
    }
}

In [ ]:
order_params_1 = copy.deepcopy(order_params)
order_params_2 = copy.deepcopy(order_params)
order_params_3 = copy.deepcopy(order_params)
order_params_4 = copy.deepcopy(order_params)
order_params_5 = copy.deepcopy(order_params)
order_params_6 = copy.deepcopy(order_params)
order_params_7 = copy.deepcopy(order_params)
order_params_8 = copy.deepcopy(order_params)
order_params_9 = copy.deepcopy(order_params)
order_params_10 = copy.deepcopy(order_params)
all_order_params = [order_params_1, order_params_2, order_params_3, order_params_4, order_params_5, order_params_6, order_params_7, order_params_8, order_params_9, order_params_10]
batched_quad_ids = [quad_ids[i:i + 100] for i in range(0, len(quad_ids), 100)]
[all_order_params[i]['products'][0].__setitem__('quad_ids', batched_quad_ids[i]) for i in range(len(batched_quad_ids))]

In [ ]:
print(len(all_order_params))

### Create and deliver order with the SDK



In [61]:
async with planet.Session() as ps:
    # The Orders API client
    client = ps.client('orders')
    # Create the order and deliver it to GCP
    await asyncio.gather(*[create_and_deliver_order(params, client) for params in all_order_params])
    
    

00:10 - order 0c517ace-9a6b-4ce6-b099-4ad7975bfa33 - state: running
00:10 - order 288c5798-510f-4823-ad88-6c456dbb42b9 - state: running


CancelledError: 

### Check your results

After a few simple steps, we have a basemap delivered to our Google Cloud Platform bucket using the Planet Python SDK.  For more information, check out the SDK docs [here](https://planet-sdk-for-python-v2.readthedocs.io/en/latest/python/sdk-guide/). 